# Using Pytorch

* Matrix Transpose

In [ ]:
import torch
a = torch.arange(6).reshape(2, 3)
torch.einsum('ij->ji', [a])

tensor([[0, 3],
        [1, 4],
        [2, 5]])

* Sum

In [ ]:
a = torch.arange(6).reshape(2, 3)
torch.einsum('ij->', [a])

tensor(15)

* Column Sum

In [ ]:
a = torch.arange(6).reshape(2, 3)
torch.einsum('ij->', [a])

tensor(15)

* Row Sum

In [ ]:
a = torch.arange(6).reshape(2, 3)
torch.einsum('ij->i', [a])

tensor([ 3, 12])

* Matrix-Vector Multiplication

In [ ]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(3)
torch.einsum('ik,k->i', [a, b])

tensor([ 5, 14])

* Matrix-Matrix Multiplication

In [ ]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(15).reshape(3, 5)
torch.einsum('ik,kj->ij', [a, b])

tensor([[ 25,  28,  31,  34,  37],
        [ 70,  82,  94, 106, 118]])

* Dot Product

In [ ]:
a = torch.arange(3)
b = torch.arange(3,6)  # -- a vector of length 3 containing [3, 4, 5]
torch.einsum('i,i->', [a, b])

tensor(14)

In [ ]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(6,12).reshape(2, 3)
torch.einsum('ij,ij->', [a, b])

tensor(145)

* Hadamard Product

In [ ]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(6,12).reshape(2, 3)
torch.einsum('ij,ij->ij', [a, b])

tensor([[ 0,  7, 16],
        [27, 40, 55]])

* Outer Product

In [ ]:
a = torch.arange(3)
b = torch.arange(3,7)  # -- a vector of length 4 containing [3, 4, 5, 6]
torch.einsum('i,j->ij', [a, b])

tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])

* 10 Batch Natrix Multiplication

In [ ]:
a = torch.randn(3,2,5)
b = torch.randn(3,5,3)
torch.einsum('ijk,ikl->ijl', [a, b])

tensor([[[-1.7533,  0.9838,  4.2215],
         [ 0.5251,  0.1514, -0.9903]],

        [[ 0.0628, -0.2901,  1.8549],
         [ 0.9902, -3.1785,  1.2734]],

        [[ 2.3475,  1.3995,  1.6970],
         [-1.0486,  3.2466, -1.6432]]])

* Tensor Contraction

In [ ]:
a = torch.randn(2,3,5,7)
b = torch.randn(11,13,3,17,5)
torch.einsum('pqrs,tuqvr->pstuv', [a, b]).shape

torch.Size([2, 7, 11, 13, 17])

* Bilinear Transformation

In [ ]:
a = torch.randn(2,3)
b = torch.randn(5,3,7)
c = torch.randn(2,7)
torch.einsum('ik,jkl,il->ij', [a, b, c])

tensor([[-0.0259,  0.2628,  1.3368, -0.1365, -1.9054],
        [-1.5134, -0.3314,  1.1292, -2.1423, -3.4076]])

* TreeQN

In [ ]:
import torch.nn.functional as F

def random_tensors(shape, num=1, requires_grad=False):
  tensors = [torch.randn(shape, requires_grad=requires_grad) for i in range(0, num)]
  return tensors[0] if num == 1 else tensors

# Parameters
# -- [num_actions x hidden_dimension]
b = random_tensors([5, 3], requires_grad=True)
# -- [num_actions x hidden_dimension x hidden_dimension]
W = random_tensors([5, 3, 3], requires_grad=True)

def transition(zl):
  # -- [batch_size x num_actions x hidden_dimension]
  return zl.unsqueeze(1) + F.tanh(torch.einsum("bk,aki->bai", [zl, W]) + b)

# Sampled dummy inputs
# -- [batch_size x hidden_dimension]
zl = random_tensors([2, 3])

transition(zl)

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([[[ 1.8000, -0.5737,  0.1176],
         [ 1.7033, -0.4210,  1.1105],
         [ 0.6457, -0.5479, -0.7458],
         [ 1.6984, -0.3482, -0.2988],
         [-0.1527,  0.5527, -0.7778]],

        [[ 2.0870, -0.5688,  3.6612],
         [ 0.0892, -0.5683,  3.6652],
         [ 1.6787,  1.1324,  1.6687],
         [ 0.7338, -0.5268,  1.6658],
         [ 0.0891,  1.1683,  1.8213]]], grad_fn=<AddBackward0>)

* Attention

In [ ]:
# Parameters
# -- [hidden_dimension]
bM, br, w = random_tensors([7], num=3, requires_grad=True)
# -- [hidden_dimension x hidden_dimension]
WY, Wh, Wr, Wt = random_tensors([7, 7], num=4, requires_grad=True)

# Single application of attention mechanism 
def attention(Y, ht, rt1):
  # -- [batch_size x hidden_dimension] 
  tmp = torch.einsum("ik,kl->il", [ht, Wh]) + torch.einsum("ik,kl->il", [rt1, Wr])
  Mt = F.tanh(torch.einsum("ijk,kl->ijl", [Y, WY]) + tmp.unsqueeze(1).expand_as(Y) + bM)
  # -- [batch_size x sequence_length]
  at = F.softmax(torch.einsum("ijk,k->ij", [Mt, w])) 
  # -- [batch_size x hidden_dimension]
  rt = torch.einsum("ijk,ij->ik", [Y, at]) + F.tanh(torch.einsum("ij,jk->ik", [rt1, Wt]) + br)
  # -- [batch_size x hidden_dimension], [batch_size x sequence_dimension]
  return rt, at

# Sampled dummy inputs
# -- [batch_size x sequence_length x hidden_dimension]
Y = random_tensors([3, 5, 7])
# -- [batch_size x hidden_dimension]
ht, rt1 = random_tensors([3, 7], num=2)

rt, at = attention(Y, ht, rt1)
at  # -- print attention weights

<ipython-input-18-edce8c55a8e9>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  at = F.softmax(torch.einsum("ijk,k->ij", [Mt, w]))


tensor([[0.2675, 0.0320, 0.5607, 0.0846, 0.0553],
        [0.0116, 0.2793, 0.0555, 0.3135, 0.3400],
        [0.3251, 0.0042, 0.6138, 0.0404, 0.0165]], grad_fn=<SoftmaxBackward0>)

In [ ]:
import numpy as np
  
  
ar1 = np.arange(9).reshape(3, 3)
ar2 = np.arange(10, 19).reshape(3, 3)

# Original Higher dimension
print(ar1)
  
print(ar2)
print("")
r = np.einsum("mk,kn", ar1, ar2)
  
# Einstein’s summation convention of 
# the said arrays
print(r)

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[10 11 12]
 [13 14 15]
 [16 17 18]]

[[ 45  48  51]
 [162 174 186]
 [279 300 321]]


In [ ]:
import torch

# Permutation of tensor
X = torch.rand((2, 3))

A = torch.einsum('ij->ji', X)
torch.transpose(X, 0, 1)  # or X.T

print(A)

tensor([[0.0124, 0.1227],
        [0.8782, 0.1298],
        [0.8623, 0.3020]])


In [ ]:
import torch

# Summation
X = torch.rand((2, 3))

a = torch.einsum('ij->', X)
torch.sum(X)

print(a) 

tensor(3.6155)


In [ ]:

import torch

X = torch.rand((2, 3))

# Row summation
a = torch.einsum('ij->i', X)
torch.sum(X, axis=1)

print(a)  # tensor([1.4088, 1.7803])

# Column summation
b = torch.einsum('ij->j', X)
torch.sum(X, axis=0)

print(b)

tensor([1.2118, 1.5576])
tensor([0.8242, 0.7641, 1.1811])


In [ ]:
import torch

# Element wise multiplication
X = torch.rand((3, 2))
Y = torch.rand((3, 2))

A = torch.einsum('ij, ij->ij', X, Y)
torch.mul(X, Y)  # or X * Y

print(A)


tensor([[0.2857, 0.1730],
        [0.1584, 0.3148],
        [0.0921, 0.7806]])


In [ ]:
import torch

# Dot product
v = torch.rand((3))
c = torch.rand((3))

a = torch.einsum('i, i->', v, c)
torch.dot(v, c)

print(a)

tensor(0.4703)


In [ ]:
import torch

# Outer product
v = torch.rand((3))
t = torch.rand((3))

A = torch.einsum('i, j->ij', v, t)
torch.outer(v, t)

print(A)

tensor([[0.5432, 0.3109, 0.0754],
        [0.1587, 0.0908, 0.0220],
        [0.5645, 0.3230, 0.0783]])


In [ ]:
import torch

# Matrix-Vector multiplication
X = torch.rand((3, 3))
y = torch.rand((1, 3))

A = torch.einsum('ij, kj->ik', X, y)
torch.mm(X, torch.transpose(y, 0, 1))  # or torch.mm(X, y.T)

print(A)

tensor([[0.6994],
        [0.6861],
        [0.3227]])


In [ ]:
import torch

# Matrix-Matrix multiplication
X = torch.arange(6).reshape(2, 3)
Y = torch.arange(12).reshape(3, 4)

A = torch.einsum('ij, jk->ik', X, Y)
torch.mm(X, Y)

print(A)


tensor([[20, 23, 26, 29],
        [56, 68, 80, 92]])


In [ ]:

import torch

# Batch matrix multiplication
X = torch.arange(24).reshape(2, 3, 4)
Y = torch.arange(40).reshape(2, 4, 5)

A = torch.einsum('ijk, ikl->ijl', X, Y)
torch.bmm(X, Y)

print(A)

tensor([[[  70,   76,   82,   88,   94],
         [ 190,  212,  234,  256,  278],
         [ 310,  348,  386,  424,  462]],

        [[1510, 1564, 1618, 1672, 1726],
         [1950, 2020, 2090, 2160, 2230],
         [2390, 2476, 2562, 2648, 2734]]])
